Проверим утверждение про затухание градиента на практике. В документации pytorch можно найти следующие функции активации (самые популярные жирным шрифтом.): 

<b>ELU</b>, Hardtanh, LeakyReLU, LogSigmoid, PReLU, <b>ReLU</b>, ReLU6, RReLU, SELU, CELU, <b>Sigmoid</b>, Softplus, Softshrink, Softsign, <b>Tanh</b>, Tanhshrink, Hardshrink.

Вам предстоит найти активацию, которая приводит к наименьшему затуханию градиента. 

Для проверки сконструируем SimpleNet, которая будет иметь внутри 3 fc-слоя, по 1 нейрону в каждом без bias'ов. Веса этих нейронов проинициализируем единицами. На вход в эту сеть будем подавать числа из нормального распределения. Сделаем 200 запусков (NUMBER_OF_EXPERIMENTS) для честного сравнения и посчитаем среднее значение градиента в первом слое. <u>Найдите</u> такую функцию, которая будет давать максимальные значения градиента в первом слое. Все функции активации нужно инициализировать с аргументами по умолчанию (пустыми скобками).


In [33]:
import torch
import numpy as np

seed = int(23)
np.random.seed(seed)
torch.manual_seed(seed)

NUMBER_OF_EXPERIMENTS = 200

class SimpleNet(torch.nn.Module):
    def __init__(self, activation):
        super().__init__()

        self.activation = activation
        self.fc1 = torch.nn.Linear(1, 1, bias=False)  # one neuron without bias
        self.fc1.weight.data.fill_(1.)  # init weight with 1
        self.fc2 = torch.nn.Linear(1, 1, bias=False)
        self.fc2.weight.data.fill_(1.)
        self.fc3 = torch.nn.Linear(1, 1, bias=False)
        self.fc3.weight.data.fill_(1.)

    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        return x

    def get_fc1_grad_abs_value(self):
        return torch.abs(self.fc1.weight.grad)

def get_fc1_grad_abs_value(net, x):
    output = net.forward(x)
    output.backward()  # no loss function. Pretending that we want to minimize output
                       # In our case output is scalar, so we can calculate backward
    fc1_grad = net.get_fc1_grad_abs_value().item()
    net.zero_grad()
    return fc1_grad

activation =  torch.nn.Hardshrink()
              # ex.: torch.nn.Tanh()

net = SimpleNet(activation=activation)

fc1_grads = []
for x in torch.randn((NUMBER_OF_EXPERIMENTS, 1)):
    fc1_grads.append(get_fc1_grad_abs_value(net, x))



In [34]:
# Проверка осуществляется автоматически, вызовом функции:
print(np.mean(fc1_grads))

0.795243504345417


Seed = 23

ELU - 0.4452307138592005

Hardtanh - 0.2993333285115659

LeakyReLU - 0.3897920662456562

LogSigmoid - 0.31556652531493457

PReLU - 0.3974751507473411

ReLU - 0.38979157449677587

ReLU6 - 0.38979157449677587

RReLU - 0.3951358215053915

SELU - 0.5648104228079319

CELU - 0.4452307138592005

Sigmoid - 0.007746891602873802

Softplus - 0.2545999674778432

Softshrink - 0.32391569316387175

Softsign - 0.06470680091530084

Tanh - 0.1663438864413183

Tanhshrink - 0.047257634050635655

Hardshrink - 0.795243504345417

Максимальное значение градиента в первом слое дает функция активации Hardshrink.
